In [250]:
import pandas as pd
import numpy as np
from math import exp, log, sqrt, pi
import matplotlib.pyplot as plt
from scipy.stats import norm
import time

In [251]:
def direction(sigma,dt):
    u = sigma*sqrt(dt)
    d = -sigma*sqrt(dt)
    return(u,d)

In [272]:
def swap(notional,fix,flt,dis,t):
    
    fixpv = notional*fix*0.01*0.5*(1+dis)**(-t)
    fltpv = (p*notional*(flt+u)*0.01*0.25+(1-p)*notional*(flt+d)*0.01*0.25)*(1+dis)**(-t)
    swap = fixpv-fltpv
    
    return swap

In [ ]:
def Intrinsic(notional,fix,flt,dis,t):
    
    fixpv = notional*fix*0.01*0.5*(1+dis)**(-t)
    fltpv = (p*notional*(flt+u)*0.01*0.25+(1-p)*notional*(flt+d)*0.01*0.25)*(1+dis)**(-t)
    value = max(0,fixpv-fltpv)
    
    return value

In [375]:
def Tree(T,T0,fix,r0,not0,sigma,dt,dis):
    
    # up and down amount in each step
    u,d = direction(sigma,dt)
    
    notional = [not0 for i in range(T*2+1)]
    for i in range(1,2*T+1):
        notional[i] = notional[i-1] + notional[i-1]*fix*0.01*0.5
    
    tfloat = [0.25*(1+2*i) for i in range(int(T*2))]
    float_layer = [int(t/dt) for t in tfloat]

    tcall = [T0+0.5*i for i in range(int((T-T0)*2))]
    call_layer = [int(t/dt) for t in tcall]
    
    # Create a 2D array for rt through all the path
    m = int(T/dt)     #total path
    rows, cols = (m, m) 
    
    rt = [[0 for i in range(cols)] for j in range(rows)] 
    rt[0][0]=r0
    
    for i in range(1,cols):    
        
        rt[0][i]=rt[0][i-1] + u
        
        for j in range(1,i+1):
            rt[j][i]=rt[j-1][i-1] + d
    
    # Create a 2D array for vt through all the path
    vt = [[0 for i in range(cols)] for j in range(rows)] 
    
    # Calc option value for layer m-1:

    for j in range(m):
        vt[j][m-1] = swap(notional[-1],fix,rt[j][m-1],dis,dt)
    
    
    for i in range(m-2,-1,-1):
        
        if i+1 in float_layer:
            if i in call_layer:
                for j in range(i+1):
                    vt[j][i] = max(0,(p*vt[j][i+1]+(1-p)*vt[j+1][i+1]-notional[int((i+1)/(0.5/dt))]*rt[j][i]*0.01*0.25)*(1+dis)**(-dt))
            else:
                for j in range(i+1):
                    vt[j][i] = ((p*vt[j][i+1]+(1-p)*vt[j+1][i+1])-notional[int((i+1)/(0.5/dt))]*rt[j][i]*0.01*0.25)*(1+dis)**(-dt)     
            
        elif i in call_layer:
            for j in range(i+1):
                vt[j][i] = max(0,(p*vt[j][i+1]+(1-p)*vt[j+1][i+1])*(1+dis)**(-dt))
        else:
            for j in range(i+1):
                vt[j][i] = (p*vt[j][i+1]+(1-p)*vt[j+1][i+1])*(1+dis)**(-dt) 
                                         
    return pd.DataFrame(vt) 
    #return pd.DataFrame(rt)

In [377]:
T = 10
T0 = 1
dt = 63/252
#dt = 0.25
r0 = 0.24
sigma = 0.06
not0 = 100
fix = 1.2
dis = 0.0008

Tree(T,T0,fix,r0,not0,sigma,dt,dis)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-0.110734,-0.067878,-0.075435,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.278900
1,0.000000,-0.033635,-0.060348,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.295803
2,0.000000,0.000000,-0.006935,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000974,0.000000,0.312706
3,0.000000,0.000000,0.000000,0.076666,0.000000,0.002550,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018796,0.001949,0.329609
4,0.000000,0.000000,0.000000,0.000000,0.153363,0.084370,0.005102,0.006906,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052491,0.035651,0.346512
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.283141,0.163671,0.094929,0.013815,0.012191,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.086186,0.069352,0.363415
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.402723,0.293565,0.176081,0.107625,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.119881,0.103054,0.380318
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.542586,0.411167,0.306060,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153576,0.136756,0.397221
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.674222,0.547166,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010318,0.000000,0.187271,0.170458,0.414124
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.801548,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.045827,0.020641,0.220966,0.204160,0.431027


In [351]:
T = 1
T0 = 0.5
dt = 63/252
#dt = 0.25
r0 = 0.24
sigma = 0.5*r0
not0 = 100
fix = 1.2
dis = 0.0008

path = int(T/dt)
tfloat = [0.25*(1+2*i) for i in range(int(T*2))]
float_layer = [int(t/dt) for t in tfloat]

tcall = [T0+0.5*i for i in range(int((T-T0)*2))]
call_layer = [int(t/dt) for t in tcall]

u,d = direction(sigma,dt)
notional = [not0 for i in range(T*2+1)]
for i in range(1,2*T+1):
    notional[i] = notional[i-1] + notional[i-1]*fix*0.01*0.5

print(path)
print(tcall)
print(call_layer)
print(tfloat)
print(float_layer)
print(notional)

4
[0.5]
[2]
[0.25, 0.75]
[1, 3]
[100, 100.6, 101.2036]


In [358]:
101.2036*0.01*0.5*((1.2*0.5-0.12*0.25)+(1.2*0.5-0.18*0.25))*1.0008**(-0.25)

0.569156452842891

In [366]:
0.5*(0.553979-100.6*0.01*0.25*0.21+0.569156-100.6*0.01*0.25*0.15)*1.0008**(-0.25)

0.5161942920987923

In [368]:
(0.516194+0.485933)/2

0.5010635

In [369]:
0.5*(0.470708-100*0.01*0.25*0.27+0.500964-100*0.01*0.25*0.21)*1.0008**(-0.25)

0.42575087535806644